This project on covid is being adaptated based on previous notebooks and similar ones that have been found on kaggle as well as our own previous developments whose hints can be found on my profiles (acutally updating some of them). References can be found within the notbook. 

Due to slightly modifications on the CORD-19 challenge database, we are still adapting the code, which is aimed to give us the results we are looking for but some parts are in need of changes. 


In [ ]:
#First we are importing the libraries we are going smeelingly use. Later on, we will import other libraries if they are necessary

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import json
import re
import string
import networkx as nx
import gc

from IPython.utils import io
#with io.capture_output() as captured:

 # !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz

from pprint import pprint
from copy import deepcopy

from collections import  Counter


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
#NATURAL LANGUAGE PROCESSING

#1. NLTK


!pip install nltk
import nltk as nlp
#nltk.download()
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer


#from rake_nltk import Rake


In [ ]:
#2. SPACY

#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz


In [ ]:
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bionlp13cg_md-0.2.4.tar.gz 

In [ ]:
!pip install scispacy

In [ ]:
import scispacy
#import en_ner_bionlp13cg_md
#import en_core_sci_lg

from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

#STOPWORDS IN ENGLISH SPACY
from spacy.lang.en.stop_words import STOP_WORDS

#TOKENIZER
from spacy.tokens import Span

#GENSIM

import gensim

# SPACY MODEL FOR LDA visualization

import pyLDAvis
import pyLDAvis.gensim

#TENSOR FLOWS
import tensorflow_hub as hub
import tensorflow as tf

#MATCHER
from spacy.matcher import Matcher 

In [ ]:
#SKLEARN

from sklearn.metrics.pairwise import cosine_similarity

#from sklearn.cluster import DBSCAN

In [ ]:
#os.listdir('../input/CORD-19-research-challenge/')
#with open('../input/CORD-19-research-challenge/metadata.readme', 'r') as f:
#   data = f.read()
#    print(data)

In [ ]:
os.listdir( '/kaggle/input/all-cleaned')

***********************


WE ARE GOING TO FOCCUS ON BIORXIV subset and on the abstract part


In [ ]:
biorxiv_clean = pd.read_csv('/kaggle/input/all-cleaned/biorxiv_clean.csv')

In [ ]:
biorxiv_clean.head(2)

checking everythin is ok with the first paper


In [ ]:
biorxiv_clean.text[0]

******************************


KEYWORD EXTRACTION. 

Now we are going to define the tasks

In [ ]:
path="../input/all-cleaned/biorxiv_clean.csv"
all_sources=pd.read_csv(path)

all_sources.isna().sum()


In [ ]:
def prepare_similarity(vectors):
    similarity=cosine_similarity(vectors)
    return similarity

def get_top_similar(sentence, sentence_list, similarity_matrix, topN):
    # find the index of sentence in list
    index = sentence_list.index(sentence)
    # get the corresponding row in similarity matrix
    similarity_row = np.array(similarity_matrix[index, :])
    # get the indices of top similar
    indices = similarity_row.argsort()[-topN:][::-1]
    return [(i,sentence_list[i]) for i in indices]

module_url = "../input/universalsentenceencoderlarge4" 
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.load(module_url)

titles=all_sources['title'].fillna("Unknown")
embed_vectors=embed(titles[:100].values)['outputs'].numpy()
sentence_list=titles.values.tolist()
sentence=titles.iloc[5]
print("Find similar research papers for :")
print(sentence)


In [ ]:
similarity_matrix=prepare_similarity(embed_vectors)
similar=get_top_similar(sentence,sentence_list,similarity_matrix,6)


In [ ]:
for sentence in similar:
    print(sentence)
print("\n")

In [ ]:
del embed_vectors,sentence_list,similarity_matrix
gc.collect()


In [ ]:
path="../input/all-cleaned/"


In [ ]:
clean_comm=pd.read_csv(path+"clean_comm_use.csv",nrows=5000)
clean_comm['source']='clean_comm'


biox = pd.read_csv(path+"biorxiv_clean.csv")
biox['source']='biorx'

all_articles=pd.concat([biox,clean_comm])

del biox,clean_comm
gc.collect()



In [ ]:
all_articles.shape


In [ ]:
#TASK AND SUBTASKS OF THE GENETICS ASPECTS

tasks= ["What is known about the virus genetics, origin and evolution", 
        "Real-time tracking of whole genomes",
        "Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences",
        "Evidence of whether farmers are infected,",
        "Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 ",
        "Experimental infections to test host range for this pathogen",
        "Animal host(s) and any evidence of continued spill-over to humans",
        "Socioeconomic and behavioral risk factors for this spill-over",
        " Sustainable risk reduction strategies"]

In [ ]:
task_df=pd.DataFrame({'title':tasks,'source':'task'})

In [ ]:
task_df.head()

In [ ]:
all_articles=pd.concat([all_articles,task_df])
all_articles.fillna("Unknown",inplace=True)

In [ ]:
sentence_list=all_articles.title.values.tolist()
embed_vectors=embed(sentence_list)['outputs'].numpy()
similarity_matrix=prepare_similarity(embed_vectors)

sentence= ["What is known about the virus genetics, origin and evolution"]

similar=get_top_similar(sentence,sentence_list,similarity_matrix,10)

for sent in similar:
    print(sent[1])



In [ ]:
for sent in similar:
    print(sent[1])


In [ ]:
#Clean and store abstracts from related articles.

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])

import re
def clean(txt):
    txt=re.sub(r'\n','',txt)
    txt=re.sub(r'\([^()]*\)','',txt)
    txt=re.sub(r'https?:\S+\sdoi','',txt)
    return txt

texts=list(map(clean,texts))
text_list=' '.join(texts)
#text_list=word_tokenize(text_list)



Keyword extraction

In [ ]:
!pip install pytextrank


In [ ]:
import logging
import pytextrank
import spacy
import sys

nlp = spacy.load("en_core_web_sm")

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger("PyTR")

#add PyTextRank into the spaCy pipeline

tr = pytextrank.TextRank(logger=None)
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)


In [ ]:
#parse the document

doc = nlp(text_list)

print("pipeline", nlp.pipe_names)
print("elapsed time: {} ms".format(tr.elapsed_time))


#examine the top-ranked phrases in the document


In [ ]:
#keywords

for phrase in doc._.phrases[:10]:
    print("{}".format(phrase.text))
#print(phrase.chunks)


Now we have it for the first task, we are going to do it for all the other tasks. We copy the model and change teh sentence for each task. 

In [ ]:

sentence= " genomes",

similar=get_top_similar(sentence,sentence_list,similarity_matrix,10)

for sent in similar:
    print(sent[1])

In [ ]:
#Clean and store abstracts from related articles.

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])

import re
def clean(txt):
    txt=re.sub(r'\n','',txt)
    txt=re.sub(r'\([^()]*\)','',txt)
    txt=re.sub(r'https?:\S+\sdoi','',txt)
    return txt

texts=list(map(clean,texts))
text_list=' '.join(texts)
#text_list=word_tokenize(text_list)


In [ ]:
nlp = spacy.load("en_core_web_sm")

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger("PyTR")

#add PyTextRank into the spaCy pipeline

tr = pytextrank.TextRank(logger=None)
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

In [ ]:
#parse the document

doc = nlp(text_list)

print("pipeline", nlp.pipe_names)
print("elapsed time: {} ms".format(tr.elapsed_time))


#examine the top-ranked phrases in the document

In [ ]:
#keywords

for phrase in doc._.phrases[:10]:
    print("{}".format(phrase.text))
#print(phrase.chunks)


**********************


*Now we are going to answer the questions: *

In [ ]:
sentence= "What do we know about virus genetics, origin, and evolution"

similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts)

In [ ]:
df = prepare_df(text_list)
draw_kg(df,c1='blue',c2='pink',c3='green')

In [ ]:
similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts)

In [ ]:
sentence= "Evidence of whether farmers are infected"

similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts)

In [ ]:
sentence= "Surveillance of mixed wildlife- livestock farms for SARS-CoV-2"

similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts)

In [ ]:
sentence= "Experimental infections to test host range for this pathogen"

similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts

In [ ]:
sentence= "Animal host(s) and any evidence of continued spill-over to humans"

similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts

In [ ]:
sentence= "Socioeconomic and behavioral risk factors for this spill-over"
similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts)

df = prepare_df(text_list)
draw_kg(df,c1='blue',c2='pink',c3='green')

In [ ]:
sentence= "Sustainable risk reduction strategies"
similar=get_top_similar(sentence,sentence_list,similarity_matrix,15)

ind,title=list(map(list,zip(*similar)))
titles=[]
texts=[]
for i in ind:
    titles.append(all_articles.iloc[i]['title'])
    texts.append(all_articles.iloc[i]['abstract'])
    
texts=list(map(clean,texts))
text_list=' '.join(texts)

df = prepare_df(text_list)
draw_kg(df,c1='blue',c2='pink',c3='green')

************************


MATCHING THE KEYWORDS WITH THE DOCUMENTS


In [ ]:
import re

file = open('wordlist.txt', 'r')

for line in file.readlines():
    if re.search('^there$', line, re.I):
        print (line)

***************************************************



NER


In [ ]:
nlp = en_ner_bionlp13cg_md.load() #LOAD MODEL

In [ ]:
sentence_list=[]
sentence_list_without_col = []
ID_list = []
ent_type_all_check = []
ent_type_all= []

for i in tqdm(range(len(abstractID[:100]))):
    ID = list(abstractID[i].keys())[0]
    text = list(abstractID[i].values())[0]
    a = tokenize.sent_tokenize(text) # Split Sentence
    
    for sent in a:
        print_flag = False
        doc = nlp(sent)
        count_label =0
        ent_type = ''
        check_dupl =[]
        sent_withcol = sent
        sent_withoutcol = sent
        
        for x in doc.ents:
            if x.text not in check_dupl:
                sent_withcol = sent_withcol.replace(x.text,f"\033[1;40m{x.text}\033[1;31;40m ({x.label_}) \033[00m")
                sent_withoutcol = sent_withoutcol.replace(x.text,f"{x.text} *{x.label_}*")
                check_dupl.append(x.text)
                print_flag =True

            if x.label_ not in ent_type:
                ent_type+= f'{x.label_}, '
                ent_type_all_check.append(x.label_)
                
        if print_flag== True:
            sentence_list.append('* '+sent_withcol)
            sentence_list_without_col.append('* '+sent)
            ent_type_all.append(ent_type)
            ID_list.append(ID)

In [ ]:
len(ID_list),len(sentence_list),len(sentence_list_without_col),len(ent_type_all)

In [ ]:
pd_all = pd.DataFrame({'Paper_ID':ID_list,'Sentence_col':sentence_list,'Sentence_wo_col':sentence_list_without_col,'NER_Key':ent_type_all})

In [ ]:
type_list = pd.Series(ent_type_all_check).unique().tolist() # Entity type

VISUALIZATION

Here is when we decide to define the Entinty as GENETICS

In [ ]:
def f(Paper_ID='all',Show_Many='100', Entity='GENETICS', Show_original=False):
    pd_all2 = pd_all.copy()[:int(Show_Many)]
    if Paper_ID != 'all':
        pd_all2 = pd_all[pd_all.Paper_ID==Paper_ID].copy()[:int(Show_Many)]
    pd_all2 = pd_all2[pd_all2['NER_Key'].str.contains(Entity)].reset_index(drop=True)
    for i in range(len(pd_all2)):
        if Show_original==True:
            print (pd_all2['Sentence_wo_col'][i])  
        else:
            print (pd_all2['Sentence_col'][i])  

In [ ]:
interact(f, Paper_ID='all',Show_Many='100',Entity=type_list, Original = False);

****************


AUTOMATIC SUMMARIZATION

The first step would be to concatenate all the text contained in the articles
Then split the text into individual sentences
In the next step, we will find vector representation (word embeddings) for each and every sentence
Similarities between sentence vectors are then calculated and stored in a matrix
The similarity matrix is then converted into a graph, with sentences as vertices and similarity scores as edges, for sentence rank calculation
Finally, a certain number of top-ranked sentences form the final summary

CONVERT ABSTRACT TO LIST

In [ ]:
df = biorxiv_clean
df = df.abstract.dropna()
data = df.values.tolist()

Find similar research papers using universalsentenceencoderlarge4

In [ ]:
%%time
#ref : https://gist.github.com/gaurav5430/8d7810495ec3f914ffb151458f352c60

'''import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
def prepare_similarity(vectors):
    similarity=cosine_similarity(vectors)
    return similarity

def get_top_similar(sentence, sentence_list, similarity_matrix, topN):
    # find the index of sentence in list
    index = sentence_list.index(sentence)
    # get the corresponding row in similarity matrix
    similarity_row = np.array(similarity_matrix[index, :])
    # get the indices of top similar
    indices = similarity_row.argsort()[-topN:][::-1]
    return [sentence_list[i] for i in indices]


meta=pd.read_csv("../input/CORD-19-research-challenge/metadata.csv")
module_url = "../input/universalsentenceencoderlarge4" 
embed = hub.load(module_url)


# Creating an empty Dataframe with column names only
simsentence = pd.DataFrame()

titles=meta['title'].fillna("Unknown")
embed_vectors=embed(titles[:5000].values)['outputs'].numpy()
sentence_list=titles.values.tolist()
for i in range(5):

    sentences=titles.iloc[i]
    #print(">>>>>>>>>>>>Using title Find similar research papers for :",sentences, "<<<<<<<<<<<<")

    similarity_matrix=prepare_similarity(embed_vectors)
    similar=get_top_similar(sentences,sentence_list,similarity_matrix,6)
    for sentence in similar:
        #print(sentence)
        simsentence = simsentence.append({'sentence': sentences, 'similar': sentence}, ignore_index=True)
        #print("\n") '''

Now let's save this new dataframe as csv file for possible further research

In [ ]:
simsentence = pd.read_csv('../input/simsentence/simsentence.csv')

The function below converts sentences to words using gensim

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

data_words = list(sent_to_words(data))

print(data_words[:1])

Build the bigram and trigram models using gensim

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

Define functions for stopwords, bigrams, trigrams and lemmatization

In [ ]:

#https://github.com/cjriggio/classifying_medical_innovation
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[1])

In [ ]:
print(data_lemmatized[:1])

Create Dictionary,Corpus and Document Frequency

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

Human readable format of corpus (term-frequency)

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
type(biorxiv_clean.csv.abstract.dropna().tolist())

In [ ]:
from gensim.summarization.summarizer import summarize
summarize(clean_noncomm_use.abstract.dropna().to_string())

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

In [ ]:
#ref : https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
from nltk.tokenize import sent_tokenize
sentences = []
for s in clean_noncomm_use.abstract.dropna():
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [ ]:
sentences[:3]

Download GloVe Word Embeddings

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

REMOVE STOPWORDS

In [ ]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

Vector Representation of Sentences

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

Similarity Matrix preparation

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
%%time
from sklearn.metrics.pairwise import cosine_similarity
for i in range(1000):
    for j in range(1000):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
            #print(sim_mat[i][j])

***********************


There are three big models for the automatic summariation methos. We are going to focus on the last one, the T5-BERT, as it was suggered in * this* notebook: https://www.kaggle.com/mobassir/mining-covid-19-scientific-papers/comments. The other two are: PageRank Algorithm & BERT algorightm. I will, eventually

Summarization Task using T5 model

In [ ]:
df1 = biorxiv_clean
df1 = df1.abstract.dropna()
df1abstracts = df.values.tolist()

len(df1abstracts)

In [ ]:
T5_PATH = 't5-base'
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH, output_past=True)
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def t5_summarize(input_text, num_beams=4, num_words=80):
    #input_text = str(input_text).replace('\n', '')
    input_text = ' '.join(input_text.split())
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt").to(device)
    summary_task = torch.tensor([[21603, 10]]).to(device)
    input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)
    summary_ids = t5_model.generate(input_tokenized,
                                    num_beams=int(num_beams),
                                    no_repeat_ngram_size=3,
                                    length_penalty=2.0,
                                    min_length=30,
                                    max_length=int(num_words),
                                    early_stopping=True)
    output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return output[0]

Summariing first 20 papers abstract of bioarvix

In [ ]:
%%time
for i in range(20):
    try:
        print('BioArvix paper  ',i + 1, " : \n" )
        print(t5_summarize(df1abstracts[i]))
        print('............................................................................\n\n\n\n')
    except:
        print('paper ',i+1 ," has LONG ABSTRACT\n\n")

In [ ]:
simsentence.head(5)

In [ ]:
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
# taken from : https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
from sentence_transformers import SentenceTransformer
import scipy.spatial

embedder = SentenceTransformer('bert-base-nli-mean-tokens')

# Corpus with example sentences
corpus = df.values.tolist()
corpus_embeddings = embedder.encode(corpus)

# Query sentences:
queries = ['Range of incubation periods for the disease in humans','risk factors of covid-19','cure for covid-19', 'antiviral covid-19 success treatment','Does smoking or pre-existing pulmonary disease increase risk of COVID-19?', 'risk of fatality among symptomatic hospitalized patients']
query_embeddings = embedder.encode(queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))

![](http://)